In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Train_cluster.csv')
data.shape

(340, 307)

In [ ]:
data.isna().sum()

Unnamed: 0    0
Train ID      0
Weekdays      0
STRT TIME     0
END TIME      0
             ..
TNP           0
KOK           0
BBQA          0
MASA          0
Cluster No    0
Length: 307, dtype: int64

In [ ]:
data.head()

,Unnamed: 0,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,...,BTTR,AXR,TMC,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No
0,0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26
2,2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10
3,3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26
4,4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26


In [ ]:
def convert_comma_sep_to_string(s):
  return ''.join(s.split(','))

data['week'] = data['Weekdays'].apply(lambda x: convert_comma_sep_to_string(x))
data.head()

,Unnamed: 0,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,...,AXR,TMC,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No,week
0,0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0000010
1,1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000010
2,2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0000100
3,3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0001000
4,4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000001


In [ ]:
def get_opposite_of_week(s):
  new = ''
  for char in s:
    if char == '0':
      new = new + '1'
    else:
      new = new + '0'
  return new

data['week_complement'] = data['week'].apply(lambda x: get_opposite_of_week(x))
data.head()

,Unnamed: 0,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,...,TMC,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No,week,week_complement
0,0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0000010,1111101
1,1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000010,1111101
2,2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0000100,1111011
3,3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0001000,1110111
4,4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000001,1111110


In [ ]:
def distance(str1, str2):
  count = 0
  for index in range(0, len(str1)):
    if str1[index] == str2[index]:
      count = count + 1
  return count

# example - distance('1111101', '0001000') = 2

In [ ]:
def find_complement(x):
  # /**
  # This will receive a pandas row.
  # For each row, a list will be calculated - list of tuples (trainId, distance)
  # */
  # print(x)
  tuple_list = []
  subset_Df = data.loc[data['Cluster No'] == x['Cluster No']]
  trainsInSameCluster = subset_Df['Train ID'].unique()
  # print(trainsInSameCluster)

  i = 0
  for train in trainsInSameCluster:
    # compare a trains week_complement with all other train weeks in same cluster.
    baseTraincomplement = x['week_complement']
    compareWithTrainWeek = data.loc[data['Train ID'] == train].iloc[0]['week']
    
    # i = i + 1 FOR DEBUGGING
    # if i < 2:
    #   print(str(baseTraincomplement) + ' train id - ' + str(train) + ' week ' + str(compareWithTrainWeek))
    
    dist = distance(baseTraincomplement, compareWithTrainWeek)
    train_tuple = (train, dist)
    tuple_list.append(train_tuple)
  
  return tuple_list

data['complement'] = data.apply(lambda x: find_complement(x), axis=1) # row by row execute match function
data.head() 

,Unnamed: 0,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,...,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No,week,week_complement,complement
0,0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0000010,1111101,"[(11045, 0), (12009, 5), (12316, 2), (13026, 2..."
1,1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000010,1111101,"[(11050, 0), (11088, 2), (11090, 2), (11092, 2..."
2,2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0000100,1111011,"[(11079, 0), (11407, 2), (12103, 2), (12107, 4..."
3,3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0001000,1110111,"[(11050, 2), (11088, 0), (11090, 2), (11092, 2..."
4,4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000001,1111110,"[(11050, 2), (11088, 2), (11090, 0), (11092, 2..."


In [ ]:
data['complement'] = data['complement'].apply(lambda full_list: sorted(full_list, key=lambda item: item[1], reverse=True))
data.head()

,Unnamed: 0,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,...,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No,week,week_complement,complement
0,0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0000010,1111101,"[(12009, 5), (13237, 4), (22911, 4), (13239, 3..."
1,1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000010,1111101,"[(12471, 5), (12298, 4), (19813, 4), (22923, 4..."
2,2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0000100,1111011,"[(15066, 5), (12107, 4), (12173, 3), (11407, 2..."
3,3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0001000,1110111,"[(12298, 4), (19813, 4), (11104, 3), (12239, 3..."
4,4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0000001,1111110,"[(19807, 5), (22903, 4), (11104, 3), (12449, 3..."


In [ ]:
data.to_csv('results.csv')

In [ ]:
## Reference Codes

distinct_cluster_list = data['Cluster No'].unique()
distinct_cluster_list

# [19,  0,  2, 24, 15,  7,  5, 11, 10, 28, 12,  6, 17, 22, 13, 25,  1,
      #  26,  3, 14, 18,  8, 20,  4, 16,  9, 21, 23, 29, 27]


for cluster in distinct_cluster_list:
  # unique_train_ids = data['Train ID'].where(data['Cluster No'] == cluster).unique()
  unique_train_ids = data.loc[data['Cluster No'] == cluster]['Train ID'].unique()
  print(unique_train_ids)

data.loc[data['Train ID'] == 11050].iloc[0]['week'] = '0100110'